<a href="https://colab.research.google.com/github/jcamilorg/prueba-agente-viajes-davi/blob/main/Prueba_Agente_Viajes_Davi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Agentes para Planificar Viajes

### Instalación de Librerias

In [3]:
!pip install langchain openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


## Diseño y desarrollo de Agentes

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
import os

# Asigna tu API key directamente a una variable de entorno temporal
os.environ["OPENAI_API_KEY"] = "tu_api_key_aquí"